In [1]:
import numpy as np
from math import log10, ceil, floor
from functools import partial
from itertools import product

## 1. Spreading

In [2]:
def compute_w(epsilon):
    return ceil(log10(1/epsilon)) + 1
    
def compute_beta(epsilon):
    return 2.3 * compute_w(epsilon)

def fine_grid_size(nonuniform_grid_size, w, upsampling_factor=2):
    # TODO fix
    sz = np.ceil(np.maximum(upsampling_factor * nonuniform_grid_size, 2*w*np.ones_like(nonuniform_grid_size)))
    return sz.astype(int)

def fine_grid_spacing(n):
    return 2*np.pi / n

def kernel(z, beta):
    return np.exp(beta * (np.sqrt(1 - np.power(z, 2)) - 1)) if abs(z) <= 1 else 0

def compute_alpha(w, n):
    return w * np.pi / n

# grid of elements of Z (signed integers) such that the kernel function is not zero
# on xi - 2 pi mi, i = 0,...,d where d is the number of dimensions
def m_grid(point, alpha):
    return tuple(
        tuple(range(ceil((xi - alpha_i)/(2*np.pi)), floor((xi + alpha_i)/(2*np.pi)) + 1)) 
            for xi, alpha_i in zip(point, alpha)
    ) 

In [3]:
# all the points in R^3 where we evaluate the function
x = [np.array([0,0,0]), np.array([0.5,0.5,0]), np.array([0.5,0.5,0.5]), np.array([-0.5,-0.5,0]),
    np.array([-0.5,0.5,0])]
N = np.array([50,50,20])
epsilon = 1.e-12

w = compute_w(epsilon)
beta = compute_beta(epsilon)
n = fine_grid_size(N, w)
h = fine_grid_spacing(n)
krn = partial(kernel, beta=beta)
alpha = compute_alpha(w,n)

print('w = {}, beta = {}'.format(w, beta))
print('N (non-uniform grid) = {}, n (uniform grid) = {}'.format(N, n))
print('h (fine grid spacing) = {}'.format(h))
print('alpha = {}'.format(alpha))

w = 13, beta = 29.9
N (non-uniform grid) = [50 50 20], n (uniform grid) = [100 100  40]
h (fine grid spacing) = [0.06283185 0.06283185 0.15707963]
alpha = [0.40840704 0.40840704 1.02101761]


In [4]:
def psi(x, alpha, kernel):
    krn = np.frompyfunc(kernel, 1, 1)
    return np.prod(krn(np.divide(x, alpha)))

# takes a point xi in input and the set of integers that make the
# |(xi - 2 pi m_i) / alpha_i| <= 1
# prt_psi accepts only a vector of coords
def psi_per(xi, prt_psi, alpha):
    # apply psi to a given combination of m1,..,md
    compute_psi = lambda ms: prt_psi(*(xi - 2*np.pi*ms))
    
    mi = m_grid(xi, alpha)
    assert all(ms for ms in mi)
    
    return sum(map(compute_psi, map(np.array, product(mi))))

In [5]:
prt_psi = partial(psi, alpha=alpha, kernel=krn)
psi_per(x[3], prt_psi, alpha=alpha)

AssertionError: 

In [ ]:
# l is a vector of fine grid indexes
# f is the vector of values of the function
#    in all the points (i.e. rows) of the matrix x
# x is a matrix whose rows are coordinates in which the function
#    is evaluated
# h is the spacing in the uniform grid
def compute_b(l, f, x, h, prt_psi_per):    
    sm = 0
    lh = np.multiply(l, h)
    for fi, xi in zip(f,x):
        sm += fi * prt_psi_per(lh - xi)
    return sm

In [ ]:
f = np.array([i for i in range(len(x))])
l = np.array([1,1,1])
prt_psi_per = partial(psi_per, prt_psi=prt_psi, alpha=alpha)

compute_b(l, f=f, x=x, h=h, prt_psi_per=prt_psi_per)

## 2. FFT

In [ ]:
prt_compute_b = partial(compute_b, f=f, x=x, h=h, prt_psi_per=prt_psi_per)
b = np.zeros(n, dtype=float)

# TODO improve performance
for cmb in map(np.array, product(*[range(ni) for ni in n])):
    b[cmb] = prt_compute_b(cmb)